In [1]:
import os
import numpy as np
import pandas as pd
import xgboost

import optuna
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error 
import joblib

logical_cores = os.cpu_count()
print(f"Number of logical CPU cores: {logical_cores}")

num_workers = max(1, logical_cores // 2)
print(f"Number of workers set to: {num_workers}")

def is_gpu_available():
    try:
        import torch
        return torch.cuda.is_available()
    except ImportError:
        return False

gpu_available = is_gpu_available()
print(f"GPU available: {gpu_available}")

print(xgboost.build_info())

Number of logical CPU cores: 16
Number of workers set to: 8
GPU available: True
{'BUILTIN_PREFETCH_PRESENT': False, 'CUDA_VERSION': [12, 4], 'DEBUG': False, 'MM_PREFETCH_PRESENT': True, 'THRUST_VERSION': [2, 3, 2], 'USE_CUDA': True, 'USE_DLOPEN_NCCL': False, 'USE_FEDERATED': False, 'USE_NCCL': False, 'USE_OPENMP': True, 'USE_RMM': False, 'libxgboost': 'C:\\Users\\ng_mi\\Anaconda\\envs\\pytorch-gpu\\Lib\\site-packages\\xgboost\\lib\\xgboost.dll'}


In [2]:
def load_or_create_ticker_df(csv_file_path):
    """
    Load the existing ticker DataFrame from a CSV file if it exists,
    otherwise create a new DataFrame with predefined column types.
    Ensure the DataFrame has the specified columns, add any missing columns,
    and rearrange the columns in alphabetical order, excluding 'Ticker_Symbol'.

    Args:
    csv_file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The loaded or newly created DataFrame.
    """
    # Define the column types
    column_types = {
        "Ticker_Symbol": str,
        "Best_Cov1D_Classification_Accuracy": float,
        "Best_Cov1D_Classification_Path": str,
        "Best_Cov1D_Regression_RMSE": float,
        "Best_Cov1D_Regression_Path": str,
        "Best_Cov1D_LSTM_Classification_Accuracy": float,
        "Best_Cov1D_LSTM_Classification_Path": str,
        "Best_Cov1D_LSTM_Regression_RMSE": float,
        "Best_Cov1D_LSTM_Regression_Path": str,
        "Best_LSTM_Classification_Accuracy": float,
        "Best_LSTM_Classification_Path": str,
        "Best_LSTM_Regression_RMSE": float,
        "Best_LSTM_Regression_Path": str,
        "Best_XGBClassifier_Classification_Accuracy": float,
        "Best_XGBClassifier_Classification_Path": str,
        "Best_XGBRegressor_Regression_RMSE": float,
        "Best_XGBRegressor_Regression_Path": str
    }


    if os.path.isfile(csv_file_path):
        # Load the existing file into a DataFrame
        ticker_df = pd.read_csv(csv_file_path)
        
        # Ensure all specified columns are present
        for column, dtype in column_types.items():
            if column not in ticker_df.columns:
                ticker_df[column] = pd.Series(dtype=dtype)
        
        # Reorder columns alphabetically, excluding 'Ticker_Symbol'
        columns = ["Ticker_Symbol"] + sorted([col for col in ticker_df.columns if col != "Ticker_Symbol"])
        ticker_df = ticker_df[columns]
    else:
        # Create a new DataFrame with the specified column types
        ticker_df = pd.DataFrame(columns=column_types.keys()).astype(column_types)
    
    return ticker_df

csv_file_path = "../ticker-best-model.csv"
ticker_df = load_or_create_ticker_df(csv_file_path)

In [3]:
os.makedirs('../feature-importances/xbclassifier', exist_ok=True)
os.makedirs('../feature-importances/xbregressor', exist_ok=True)
os.makedirs('../models/xgboost/xbclassifier', exist_ok=True)
os.makedirs('../models/xgboost/xbregressor', exist_ok=True)

path = '../data'

ticker_list = []

if os.path.exists(path):
    ticker_list = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.csv')]


In [4]:
def preprocess_data(df):
    if df.isna().sum().sum() > 0 or df.isin([float('inf'), float('-inf')]).sum().sum() > 0:
        df = df.replace([float('inf'), float('-inf')], float('nan')).dropna()

    df = df.dropna()

    columns_to_drop = [
        'NEXT_DAY_CLOSEPRICE', 'DAILY_CLOSEPRICE_CHANGE', 'CLOSEPRICE_DIRECTION',
        'DAILY_MIDPRICE', 'NEXT_DAY_MIDPRICE', 'DAILY_MIDPRICE_CHANGE', 'MIDPRICE_DIRECTION', 'Date'
    ]
    X = df.drop(columns=columns_to_drop)
    y_classifier = (df['DAILY_CLOSEPRICE_CHANGE'] > 0).astype(int)
    y_regressor = df['DAILY_CLOSEPRICE_CHANGE']

    return X, y_classifier, y_regressor

In [5]:
def process_xbclassifier(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'use_label_encoder': False,
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        accuracy = accuracy_score(y_valid, preds)
        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    best_model = XGBClassifier(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'

    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBClassifier_Classification_Accuracy'].values[0]
        if pd.isnull(current_score) or study.best_value > current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBClassifier_Classification_Accuracy', 'Best_XGBClassifier_Classification_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with accuracy: {study.best_value}")
             # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model accuracy: {current_score} is better for {ticker} than accuracy: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBClassifier_Classification_Accuracy': [study.best_value], 'Best_XGBClassifier_Classification_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with accuracy: {study.best_value}")

        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")

    return ticker_df


def process_xbregressor(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        rmse = root_mean_squared_error (y_valid, preds)
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_model = XGBRegressor(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'
    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBRegressor_Regression_RMSE'].values[0]
        if pd.isnull(current_score) or study.best_value < current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBRegressor_Regression_RMSE', 'Best_XGBRegressor_Regression_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with RMSE: {study.best_value}")

            # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model MSE: {current_score} is better for {ticker} than MSE: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBRegressor_Regression_RMSE': [study.best_value], 'Best_XGBRegressor_Regression_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with RMSE: {study.best_value}")
        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")
   
    
    return ticker_df

In [6]:
for ticker in ticker_list:
    dataframe = pd.read_csv(f"../data/{ticker}.csv")
    X, y_classifier, y_regressor = preprocess_data(dataframe)
    ticker_df = process_xbclassifier(X, y_classifier, gpu_available, ticker, ticker_df, csv_file_path)
    ticker_df = process_xbregressor(X, y_regressor, gpu_available, ticker, ticker_df, csv_file_path)


[I 2024-09-05 19:45:39,318] A new study created in memory with name: no-name-137ccad4-5410-461f-908e-dd9f7fa12769
[I 2024-09-05 19:45:39,585] Trial 0 finished with value: 0.5757575757575758 and parameters: {'n_estimators': 452, 'max_depth': 9, 'learning_rate': 0.16940720922259064, 'subsample': 0.9932033392777627, 'colsample_bytree': 0.9050391213102388, 'gamma': 2.0752782937705487, 'lambda': 0.655556045137223, 'alpha': 6.029700395814915e-08}. Best is trial 0 with value: 0.5757575757575758.
[I 2024-09-05 19:45:39,758] Trial 1 finished with value: 0.6060606060606061 and parameters: {'n_estimators': 674, 'max_depth': 4, 'learning_rate': 0.2921748141658441, 'subsample': 0.821751633375739, 'colsample_bytree': 0.7717178872101871, 'gamma': 0.49051286201980016, 'lambda': 0.002195822831116554, 'alpha': 0.37267467110798264}. Best is trial 1 with value: 0.6060606060606061.
[I 2024-09-05 19:45:39,964] Trial 2 finished with value: 0.6363636363636364 and parameters: {'n_estimators': 162, 'max_depth':

Previous model accuracy: 0.7575757575757576 is better for CL=F than accuracy: 0.7272727272727273


[I 2024-09-05 19:46:22,451] Trial 0 finished with value: 1.3940441110328572 and parameters: {'n_estimators': 901, 'max_depth': 6, 'learning_rate': 0.0919840489294419, 'subsample': 0.5607662331571992, 'colsample_bytree': 0.8161701949477201, 'gamma': 0.6165960260553738, 'lambda': 9.1140314464196e-08, 'alpha': 0.02383038227102634}. Best is trial 0 with value: 1.3940441110328572.
[I 2024-09-05 19:46:22,973] Trial 1 finished with value: 1.396980380506691 and parameters: {'n_estimators': 512, 'max_depth': 4, 'learning_rate': 0.03556587046897561, 'subsample': 0.9811751859366818, 'colsample_bytree': 0.8807204815522104, 'gamma': 0.18281446449336047, 'lambda': 0.08133376901695974, 'alpha': 0.6833903246242985}. Best is trial 0 with value: 1.3940441110328572.
[I 2024-09-05 19:46:23,458] Trial 2 finished with value: 1.3860824427101621 and parameters: {'n_estimators': 930, 'max_depth': 10, 'learning_rate': 0.07952668014114153, 'subsample': 0.6639979304467869, 'colsample_bytree': 0.7460201084133182, 

Previous model MSE: 1.212603462635131 is better for CL=F than MSE: 1.2237183351302277


[I 2024-09-05 19:47:04,488] Trial 0 finished with value: 0.5454545454545454 and parameters: {'n_estimators': 606, 'max_depth': 5, 'learning_rate': 0.06830676371694962, 'subsample': 0.9317000208225839, 'colsample_bytree': 0.9086028249104345, 'gamma': 2.002097554152688, 'lambda': 3.7896453972257713e-07, 'alpha': 2.119155508017364e-05}. Best is trial 0 with value: 0.5454545454545454.
[I 2024-09-05 19:47:04,804] Trial 1 finished with value: 0.6363636363636364 and parameters: {'n_estimators': 487, 'max_depth': 10, 'learning_rate': 0.20414588148483076, 'subsample': 0.7317468426712943, 'colsample_bytree': 0.9047290893942266, 'gamma': 4.823500657327131, 'lambda': 2.6815444569609235e-05, 'alpha': 6.991632183827833e-08}. Best is trial 1 with value: 0.6363636363636364.
[I 2024-09-05 19:47:05,113] Trial 2 finished with value: 0.45454545454545453 and parameters: {'n_estimators': 232, 'max_depth': 10, 'learning_rate': 0.20753150968943931, 'subsample': 0.9504991130593894, 'colsample_bytree': 0.962738

...

Previous model MSE: 0.0015891294306483 is better for SGDUSD=X than MSE: 0.001589129450666035


[I 2024-09-05 19:50:17,008] Trial 0 finished with value: 0.4857142857142857 and parameters: {'n_estimators': 241, 'max_depth': 4, 'learning_rate': 0.1067047865486293, 'subsample': 0.5982602732374898, 'colsample_bytree': 0.6066878796359461, 'gamma': 1.8642039552867118, 'lambda': 1.0059638795935707e-07, 'alpha': 1.354692996782157e-06}. Best is trial 0 with value: 0.4857142857142857.
[I 2024-09-05 19:50:17,477] Trial 1 finished with value: 0.6285714285714286 and parameters: {'n_estimators': 943, 'max_depth': 8, 'learning_rate': 0.11816109615536992, 'subsample': 0.8185664085037295, 'colsample_bytree': 0.7798917495710351, 'gamma': 1.1052727045515254, 'lambda': 3.699623336536476e-05, 'alpha': 1.473600539356095e-07}. Best is trial 1 with value: 0.6285714285714286.
[I 2024-09-05 19:50:17,795] Trial 2 finished with value: 0.4857142857142857 and parameters: {'n_estimators': 850, 'max_depth': 6, 'learning_rate': 0.20787906876715725, 'subsample': 0.708035447374064, 'colsample_bytree': 0.6200181272

Previous model accuracy: 0.7142857142857143 is better for USDSGD=X than accuracy: 0.7142857142857143


[I 2024-09-05 19:51:06,656] Trial 0 finished with value: 0.002834276801707513 and parameters: {'n_estimators': 950, 'max_depth': 8, 'learning_rate': 0.15602541155699287, 'subsample': 0.8392959855410655, 'colsample_bytree': 0.6189237866598452, 'gamma': 4.123304653059282, 'lambda': 0.00021062477665412127, 'alpha': 0.002847201815238049}. Best is trial 0 with value: 0.002834276801707513.
[I 2024-09-05 19:51:06,900] Trial 1 finished with value: 0.0028392896750757415 and parameters: {'n_estimators': 206, 'max_depth': 4, 'learning_rate': 0.09145059615165563, 'subsample': 0.7402356606576506, 'colsample_bytree': 0.8077205242617429, 'gamma': 0.023666682673866846, 'lambda': 1.2391738760514503e-05, 'alpha': 0.13969102227483204}. Best is trial 0 with value: 0.002834276801707513.
[I 2024-09-05 19:51:07,172] Trial 2 finished with value: 0.002832286374279713 and parameters: {'n_estimators': 816, 'max_depth': 8, 'learning_rate': 0.12636561411867825, 'subsample': 0.7399197713615923, 'colsample_bytree': 

Best model for USDSGD=X saved with RMSE: 0.0028284929855295895
Feature importances for USDSGD=X saved.
